In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from deepface import DeepFace
import torch
import matplotlib
matplotlib.use('TkAgg')  # Set backend before importing pyplot
import matplotlib.pyplot as plt
import time

# Funcții de preprocesare a imaginii
def apply_clahe(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv[:, :, 2] = clahe.apply(hsv[:, :, 2])
    img_clahe = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return img_clahe

def apply_gamma_correction(img, gamma=1.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(256)]).astype("uint8")
    img_gamma = cv2.LUT(img, table)
    return img_gamma

# Verifică dacă GPU-ul este disponibil
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Initializează detectorul MTCNN fără puncte cheie
detector = MTCNN()

# Inițializează captura video
cap = cv2.VideoCapture(0)  # 0 pentru camera implicită
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Definirea emoțiilor în engleză (conform FER Plus)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Încărcarea modelului de detectare a emoțiilor
model_path = 'model_emotion_vggish.h5'  # Asigură-te că ai modelul în acest path
try:
    model = load_model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

print("Starting video stream. Press 'q' to exit.")

# Initialize Matplotlib pentru plotare în timp real
plt.ion()
fig, ax = plt.subplots()
emotion_counts = {label: 0 for label in emotion_labels}
bars = ax.bar(emotion_labels, [0]*len(emotion_labels), color='skyblue')
ax.set_ylim(0, 10)  # Limită inițială a axei Y
ax.set_ylabel('Count')
ax.set_title('Emotion Counts Over Time')
plt.show()

frame_counter = 0
start_time = time.time()

def get_largest_face(faces):
    if not faces:
        return None
    largest_face = max(faces, key=lambda face: face['box'][2] * face['box'][3])
    return largest_face

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame from camera.")
        break

    # Aplică preprocesarea: CLAHE și corecție gamma
    processed_frame = apply_clahe(frame)
    processed_frame = apply_gamma_correction(processed_frame, gamma=1.5)

    # Detectare fețe cu MTCNN
    faces = detector.detect_faces(processed_frame)
    largest_face = get_largest_face(faces)

    # Creare mască pentru a blura fundalul
    mask = np.zeros_like(processed_frame)
    if largest_face:
        x, y, w, h = largest_face['box']
        x, y = max(0, x), max(0, y)
        w, h = min(w, processed_frame.shape[1] - x), min(h, processed_frame.shape[0] - y)
        cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 255, 255), -1)

    # Aplicare blur pe fundal
    blurred_frame = cv2.GaussianBlur(processed_frame, (21, 21), 0)
    # Combina blurred_frame și processed_frame folosind masca
    combined_frame = np.where(mask == np.array([255, 255, 255]), processed_frame, blurred_frame)

    current_time = time.time()
    if current_time - start_time >= 1:
        if largest_face:
            try:
                face_roi_original = combined_frame[y:y + h, x:x + w].copy()
                face_gray = cv2.cvtColor(face_roi_original, cv2.COLOR_BGR2GRAY)
                face_eq = cv2.equalizeHist(face_gray)
                face_eq = cv2.GaussianBlur(face_eq, (3, 3), 0)
                face_roi = cv2.resize(face_eq, (48, 48))
                face_roi = face_roi.astype('float32') / 255.0
                face_roi = np.expand_dims(face_roi, axis=-1)
                face_roi = np.expand_dims(face_roi, axis=0)

                # Predicție emoție folosind modelul antrenat
                predictions = model.predict(face_roi)[0]
                if not isinstance(predictions, np.ndarray):
                    predictions = np.array(predictions)
                predictions = predictions / np.sum(predictions)

                # Predicție emoție folosind DeepFace
                face_roi_rgb = cv2.cvtColor(face_roi_original, cv2.COLOR_BGR2RGB)
                deepface_result = DeepFace.analyze(face_roi_rgb, actions=['emotion'], enforce_detection=False)

                # Extrage probabilitățile emoțiilor din DeepFace
                if isinstance(deepface_result, list):
                    if len(deepface_result) > 0:
                        deepface_emotions = deepface_result[0].get('emotion', {})
                    else:
                        deepface_emotions = {}
                elif isinstance(deepface_result, dict):
                    deepface_emotions = deepface_result.get('emotion', {})
                else:
                    deepface_emotions = {}

                # Maparea etichetelor DeepFace la cele din modelul tău personalizat
                deepface_emotions = {k.capitalize(): v for k, v in deepface_emotions.items()}
                deepface_mapped = {
                    'Angry': deepface_emotions.get('Angry', 0.0),
                    'Disgust': deepface_emotions.get('Disgust', 0.0),
                    'Fear': deepface_emotions.get('Fear', 0.0),
                    'Happy': deepface_emotions.get('Happy', 0.0),
                    'Sad': deepface_emotions.get('Sad', 0.0),
                    'Surprise': deepface_emotions.get('Surprise', 0.0),
                    'Neutral': deepface_emotions.get('Neutral', 0.0)
                }

                deepface_predictions = np.array([deepface_mapped[label] for label in emotion_labels])
                if np.sum(deepface_predictions) > 0:
                    deepface_predictions = deepface_predictions / np.sum(deepface_predictions)
                else:
                    deepface_predictions = np.zeros(len(emotion_labels))

                # Calcularea mediei probabilităților
                combined_predictions = (predictions + deepface_predictions) / 2
                final_max_index = np.argmax(combined_predictions)
                final_emotion = emotion_labels[final_max_index]
                emotion_counts[final_emotion] += 1

                # Desenare dreptunghi pe față fără acuratețe
                color = (255, 0, 0)  # Albastru
                cv2.rectangle(combined_frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(combined_frame, final_emotion, (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # Actualizare grafic
                for bar, label in zip(bars, emotion_labels):
                    bar.set_height(emotion_counts[label])
                max_count = max(emotion_counts.values()) if emotion_counts.values() else 10
                ax.set_ylim(0, max(max_count + 5, 10))
                fig.canvas.draw()
                fig.canvas.flush_events()
                plt.pause(0.001)

            except Exception as e:
                print("Error processing face:", e)
                pass

        start_time = current_time

    cv2.imshow('Original Frame', frame)
    cv2.imshow('Processed Frame - Blurred Background and Emotion', combined_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Using device: cpu


Model loaded successfully.
Starting video stream. Press 'q' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━